In [9]:
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns

In [10]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.12.0


In [11]:
dataset = pd.read_csv('auto-mpg.csv')
dataset.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model-year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


In [12]:
dataset.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      2
weight          0
acceleration    0
model-year      0
dtype: int64

In [13]:
dataset = dataset.dropna()

In [14]:
# Listamo jedinstvene vrednosti u koloni (na 2 razlicita nacina)
print(dataset["Origin"].unique())

KeyError: 'Origin'

In [15]:
dataset['Origin'] = dataset['Origin'].map({1: 'USA', 2: 'Europe', 3: 'Japan'})

KeyError: 'Origin'

In [16]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model-year
393,27.0,4,140.0,86.0,2790,15.6,82
394,44.0,4,97.0,52.0,2130,24.6,82
395,32.0,4,135.0,84.0,2295,11.6,82
396,28.0,4,120.0,79.0,2625,18.6,82
397,31.0,4,119.0,82.0,2720,19.4,82


In [17]:
# Uzimamo 80% podataka kao podatke za obucavanje
train_dataset = dataset.sample(frac=0.8,random_state=0)

# Ostalih 20% koristimo kao podatke za testiranje
test_dataset = dataset.drop(train_dataset.index)

In [19]:
normed_train_dataset = (train_dataset - test_dataset.mean())/ test_dataset.std()
normed_train_dataset

,mpg,cylinders,displacement,horsepower,weight,acceleration,model-year
302,1.289723,-0.841175,-0.801238,-0.805025,-0.897056,-0.208947,0.717784
321,1.007042,-0.841175,-0.772949,-0.680618,-0.764613,-0.092428,0.987808
320,1.596985,-0.841175,-0.669220,-0.257633,-0.569979,-0.170107,0.987808
66,-0.861112,1.470228,1.075300,1.185491,0.855800,-1.529492,-1.172381
60,-0.492397,-0.841175,-0.471194,-0.307396,-0.599923,1.577673,-1.172381
...,...,...,...,...,...,...,...
281,-0.516978,0.314526,0.094597,-0.431803,0.070354,1.072758,0.717784
229,-0.984017,1.470228,1.980565,1.931934,1.486920,-1.684850,0.177737
150,0.245032,-0.841175,-0.772949,-0.232752,-0.619501,0.024090,-0.632334
145,0.982461,-0.841175,-1.008695,-1.028958,-1.066353,1.383475,-0.632334


In [79]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

In [80]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                512       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 4,737
Trainable params: 4,737
Non-trainable params: 0
_________________________________________________________________


In [81]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

Epoch 1/1000


ValueError: in user code:

    File "/home/wei/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/wei/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/wei/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/wei/.local/lib/python3.10/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/wei/.local/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/wei/.local/lib/python3.10/site-packages/keras/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    Va

In [ ]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model